# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("clean_cities.csv", encoding="utf-8")
cities_df.head()


,City,Cloudiness,Country,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph)
0,aracataca,26.0,CO,85.0,10.59,-74.19,81.00,1.72
1,lakes entrance,60.0,AU,60.0,-37.88,147.98,59.05,6.33
2,punta arenas,0.0,CL,86.0,-53.15,-70.92,25.25,4.70
3,pouebo,20.0,NC,53.0,-20.40,164.57,70.92,7.78
4,esperance,40.0,AU,87.0,-33.87,121.90,48.20,19.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities_df["Humidity"]
max_humidity = max(humidity)
locations = cities_df[["Latitude", "Longitude"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_df2 = cities_df.loc[(cities_df["Temperature"] > 60) & 
                           (cities_df["Temperature"] < 75) & 
                           (cities_df["Cloudiness"] == 0) &
                           (cities_df["Wind Speed (mph)"] <= 10)&
                           (cities_df["Humidity"] <= 85)]
cities_df2.head()

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph)
30,ballina,0.0,AU,77.0,-28.87,153.57,63.00,3.36
35,lebedinyy,0.0,RU,43.0,58.52,125.52,72.48,9.01
46,haines junction,0.0,CA,30.0,60.75,-137.51,66.47,9.51
69,port macquarie,0.0,AU,62.0,-31.43,152.92,64.00,1.01
97,trinidad,0.0,BO,74.0,-14.83,-64.90,73.99,1.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cities_df2
parameters = { "radius": 5000, 
              "types": "lodging", 
              "keyword": "hotel",
              "key": g_key}
for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    parameters["location"]=f"{latitude},{longitude}"

    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=parameters)
    formatted_response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"]=formatted_response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Location not found")
hotel_df

/Users/juanferc/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/juanferc/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Location not found
Location not found
Location not found
Location not found


,City,Cloudiness,Country,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph),Hotel Name
30,ballina,0.0,AU,77.0,-28.87,153.57,63.00,3.36,Ramada Hotel & Suites by Wyndham Ballina Byron
35,lebedinyy,0.0,RU,43.0,58.52,125.52,72.48,9.01,NaN
46,haines junction,0.0,CA,30.0,60.75,-137.51,66.47,9.51,Raven's Rest Inn
69,port macquarie,0.0,AU,62.0,-31.43,152.92,64.00,1.01,Sails Port Macquarie by Rydges
97,trinidad,0.0,BO,74.0,-14.83,-64.90,73.99,1.92,Hotel Río Ibare
108,abu samrah,0.0,SY,67.0,35.30,37.18,69.44,8.21,NaN
111,ustikolina,0.0,BA,80.0,43.58,18.79,63.77,1.72,Brajlovic Ranch
139,ravar,0.0,IR,20.0,31.27,56.81,74.79,3.11,Hotel sperlous
144,himare,0.0,AL,83.0,40.10,19.74,72.00,3.42,Sea View Hotel Himare
189,luderitz,0.0,NaN,21.0,-26.65,15.16,62.60,6.93,LÜDERITZ NEST HOTEL


In [7]:
clean_df = hotel_df.dropna()
clean_df.head()

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph),Hotel Name
30,ballina,0.0,AU,77.0,-28.87,153.57,63.00,3.36,Ramada Hotel & Suites by Wyndham Ballina Byron
46,haines junction,0.0,CA,30.0,60.75,-137.51,66.47,9.51,Raven's Rest Inn
69,port macquarie,0.0,AU,62.0,-31.43,152.92,64.00,1.01,Sails Port Macquarie by Rydges
97,trinidad,0.0,BO,74.0,-14.83,-64.90,73.99,1.92,Hotel Río Ibare
111,ustikolina,0.0,BA,80.0,43.58,18.79,63.77,1.72,Brajlovic Ranch


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))